<a href="https://colab.research.google.com/github/supriyag123/PHD_Pub/blob/main/DGRNet%20-%20Last%20Step%20_Comparison%20Results.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [180]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
import tensorflow as tf
import os
import math
import plotly.graph_objects as go
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Dropout, RepeatVector, TimeDistributed, Input
from keras.models import Model
from keras import backend as K
from tensorflow.keras.optimizers import *
from tensorflow.keras.callbacks import EarlyStopping
import keras.backend as K
from keras.callbacks import Callback
import plotly
import plotly.express as px # for data visualization
import seaborn as sns
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split
from sklearn.ensemble import IsolationForest
import numpy as np
import scipy.stats as stats
from numpy import array
from numpy import hstack
from numpy import mean
from numpy import std
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LassoCV, MultiTaskLassoCV
from sklearn.model_selection import RepeatedKFold
from sklearn.model_selection import TimeSeriesSplit
from math import sqrt
from sklearn.linear_model import LinearRegression

#-----------------------------------------------------GET DATA - X, Predicted Wndow from MLP, and Derived Labels, for the entire dataset that was gennerated through VAE-----------------------------------

#y_pred = np.loadtxt(r'/content/drive/MyDrive/PHD/2024/MLPOutput/predicted_COSW.csv')
#y_true = np.loadtxt(r'/content/drive/MyDrive/PHD/2024/MLPOutput/Calculated_label_COSW.csv')
#x_orig = np.loadtxt(r'/content/drive/MyDrive/PHD/2024/MLPOutput/whole_data.csv')

y_pred = np.loadtxt(r'/content/drive/MyDrive/PHD/2024/MLPOutput/predicted_window.csv')
#np.savetxt(r'/content/drive/MyDrive/PHD/2024/MLPOutput/derived_window_label.csv',y_test_true)
x_orig = np.loadtxt(r'/content/drive/MyDrive/PHD/2024/MLPOutput/test_data.csv')

n_features = 6
window_size = int(x_orig.shape[1]/n_features)
x = x_orig.reshape((x_orig.shape[0],window_size,n_features))


#------------------------------------------------------------------------------------------------------

# split a multivariate long sequence into small sequences, and create target value 1 step in right
def split_sequences(sequences, n_steps):
 X, y = list(), list()
 for i in range(len(sequences)):
 # find the end of this pattern
  end_ix = i + n_steps
 # check if we are beyond the dataset
  if end_ix > len(sequences)-1:
    break
 # gather input and output parts of the pattern
  seq_x, seq_y = sequences[i:end_ix, :], sequences[end_ix, :]
  X.append(seq_x)
  y.append(seq_y)
 return array(X), array(y)

#-----------------------------------------------CASES - pot window values and pick a few fixed window sizes -------------------------------------------

plt.figure(figsize=(15,6))
plt.subplot(1,2,1)
plt.title("Distribution before Transformation", fontsize=15)
sns.histplot(y_pred, kde=True, color="red")
plt.subplot(1,2,2)

fx_window1 = 5
fx_window2 = 10
fx_window3 = 15
fx_window4 = 20
fx_window5 = 25
#-------------------------Take each lng sequence and loop. Send it to split_sequence with a window size, get smaller slices and target values back.
#-------------------------Then,

def get_mse(sequences,window):
  x_1,y_1 =  split_sequences( sequences,window)
  x_1=x_1.reshape(x_1.shape[0],x_1.shape[1]*x_1.shape[2])
  tscv = TimeSeriesSplit(n_splits=n_fold-1)
   # regressor = MultiTaskLassoCV(alphas=np.arange(0, 1, 0.01), cv=tscv, n_jobs=-1)
  regressor = LinearRegression()
  maxval2 = x_1.shape[0]
  count_train2 = int(math.ceil(0.7*maxval2))
  x_train = x_1[0:count_train2]
  x_test = x_1[count_train2:]
  y_train = y_1[0:count_train2]
  y_test = y_1[count_train2:]
  regressor.fit(x_train, y_train)
  ypred = regressor.predict(x_test)
  mse = mean_squared_error(y_test, ypred)
  return mse

rmse_list1 = list()
rmse_list2 = list()
rmse_list3 = list()
rmse_list4 = list()
rmse_list5 = list()
rmse_list_dgr = list()

n_fold = 6


for i in range(x.shape[0]) :

#for i in range(60000,65000) :
  print(i)
  print(x[i].shape)
  mse1 = get_mse(x[i],fx_window1)
  rmse_list1.append(mse1)
  mse2 = get_mse(x[i],fx_window2)
  rmse_list2.append(mse2)
  mse3 = get_mse(x[i],fx_window3)
  rmse_list3.append(mse3)
  mse4 = get_mse(x[i],fx_window4)
  rmse_list4.append(mse4)
  mse5 = get_mse(x[i],fx_window5)
  rmse_list5.append(mse5)
  mse_dgr = get_mse(x[i],int(y_pred[i]))
  rmse_list_dgr.append(mse_dgr)


MSE1 = mean(rmse_list1)
MSE2 = mean(rmse_list2)
MSE3 = mean(rmse_list3)
MSE4 = mean(rmse_list4)
MSE5 = mean(rmse_list5)
MSE_DGR = mean(rmse_list_dgr)

RMSE1 = sqrt(MSE1)
RMSE2 = sqrt(MSE2)
RMSE3 = sqrt(MSE3)
RMSE4 = sqrt(MSE4)
RMSE5 = sqrt(MSE5)
RMSE_DGR = sqrt(MSE_DGR)

print (RMSE1,RMSE2,RMSE3,RMSE4,RMSE5, RMSE_DGR)





0
(30, 6)
1
(30, 6)
2
(30, 6)
3
(30, 6)
4
(30, 6)
5
(30, 6)
6
(30, 6)
7
(30, 6)
8
(30, 6)
9
(30, 6)
10
(30, 6)
11
(30, 6)
12
(30, 6)
13
(30, 6)
14
(30, 6)
15
(30, 6)
16
(30, 6)
17
(30, 6)
18
(30, 6)
19
(30, 6)
20
(30, 6)
21
(30, 6)
22
(30, 6)
23
(30, 6)
24
(30, 6)
25
(30, 6)
26
(30, 6)
27
(30, 6)
28
(30, 6)
29
(30, 6)
30
(30, 6)
31
(30, 6)
32
(30, 6)
33
(30, 6)
34
(30, 6)
35
(30, 6)
36
(30, 6)
37
(30, 6)
38
(30, 6)
39
(30, 6)
40
(30, 6)
41
(30, 6)
42
(30, 6)
43
(30, 6)
44
(30, 6)
45
(30, 6)
46
(30, 6)
47
(30, 6)
48
(30, 6)
49
(30, 6)
50
(30, 6)
51
(30, 6)
52
(30, 6)
53
(30, 6)
54
(30, 6)
55
(30, 6)
56
(30, 6)
57
(30, 6)
58
(30, 6)
59
(30, 6)
60
(30, 6)
61
(30, 6)
62
(30, 6)
63
(30, 6)
64
(30, 6)
65
(30, 6)
66
(30, 6)
67
(30, 6)
68
(30, 6)
69
(30, 6)
70
(30, 6)
71
(30, 6)
72
(30, 6)
73
(30, 6)
74
(30, 6)
75
(30, 6)
76
(30, 6)
77
(30, 6)
78
(30, 6)
79
(30, 6)
80
(30, 6)
81
(30, 6)
82
(30, 6)
83
(30, 6)
84
(30, 6)
85
(30, 6)
86
(30, 6)
87
(30, 6)
88
(30, 6)
89
(30, 6)
90
(30, 6)
91
(30, 6

ValueError: setting an array element with a sequence. The requested array has an inhomogeneous shape after 1 dimensions. The detected shape was (30,) + inhomogeneous part.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')